In [ ]:
import undetected_chromedriver.v2 as uc
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from importlib.machinery import SourceFileLoader
import time
from selenium import webdriver
import json

In [ ]:
base_url = 'https://www.boots.com/'
driver = uc.Chrome()
driver.get(base_url)

In [ ]:
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, 'onetrust-accept-btn-handler')))
accept_cookies = driver.find_elements(By.ID, 'onetrust-accept-btn-handler')
if len(accept_cookies) > 0:
    accept_cookies[0].click()

In [ ]:
conf = {}
conf['country'] = 'United Kingdom'
conf['retailer'] = 'Boots'
conf['parsing_config'] = './retailer_config/bootscom_parsing.json'
conf['gcloud_path'] = 'crawl_data/United Kingdom/Boots/'
page_configs = []

In [ ]:
def getFilename(conf):
    file_name = page_config['country'] + '_' + page_config['retailer'] + '_' + page_config['department'] + '_' + page_config['category'] + '_' + page_config['page']
    for k, v in json.load(open('config/encoding.json', 'r')).items():
        file_name = file_name.replace(k, v)
        file_name = file_name.replace('\\', '')
    return file_name


In [ ]:
departments_container = driver.find_element(By.CLASS_NAME, 'topLevelMenuListItem')
action = ActionChains(driver)
action.move_to_element(departments_container).perform()
departments = departments_container.find_elements(By.CLASS_NAME, 'departmentMenuListItem')
departments_texts = []
for department in departments:
    action = ActionChains(driver)
    action.move_to_element(department).perform()
    time.sleep(1)
    department_name = department.find_element(By.CLASS_NAME,'menuItemLabel').text.strip()
    department_page = department.find_elements(By.CLASS_NAME, 'viewAllHeader')
    if not len(department_page):
        continue
    print(department_name + ' | ' + department_page[0].text.strip() + ' | ' + department_page[0].text.strip())
    print(department_page[0].get_attribute('href'))
    print('********')
    page_config = conf.copy()     
    page_config['department'] = department_name
    page_config['category'] = department_page[0].text.strip()
    page_config['page'] = department_page[0].text.strip()
    page_config['page_url'] = department_page[0].get_attribute('href')
    page_config['file_name'] = getFilename(page_config)
    page_configs.append(page_config)
#     print(page_config)
    
    categories = department.find_elements(By.CLASS_NAME, 'categoryMenuListItem')
    for cat in categories[1:]:
        action = ActionChains(driver)
        action.move_to_element(cat).perform()
        time.sleep(2)
        category_name = cat.find_element(By.CLASS_NAME, 'categoryLinkItem').text
        
        if len(cat.find_elements(By.CLASS_NAME, 'viewAllHeader')):
            category_page = cat.find_element(By.CLASS_NAME, 'viewAllHeader')
            print(department_name + ' | ' + category_name + ' | ' + category_page.text)
            print(category_page.get_attribute('href'))
            print('********')
            page_config = conf.copy()            
            page_config['department'] = department_name
            page_config['category'] = category_name
            page_config['page'] = category_page.text
            page_config['page_url'] = category_page.get_attribute('href')
            page_config['file_name'] = getFilename(page_config)
            page_configs.append(page_config)
#             print(page_config)
            
            
            pages = cat.find_elements(By.CLASS_NAME, 'menuLink')
            for page in pages[1:]:
                try:
                    action = ActionChains(driver)
                    action.move_to_element(page).perform()
                    time.sleep(0.5)
                except Exception as e:
                    x=1
                print(department_name, ' | ', category_name, ' | ', page.text )
                print(page.get_attribute('href'))
                print('********')
                page_config = conf.copy()            
                page_config['department'] = department_name
                page_config['category'] = category_name
                page_config['page'] = page.text
                page_config['page_url'] = page.get_attribute('href')
                page_config['file_name'] = getFilename(page_config)
                page_configs.append(page_config)
#                 print(page_config)
        else:
            print(department_name + ' | ' + category_name + ' | ' + category_name)
            print(cat.find_element(By.CLASS_NAME, 'categoryLinkItem').get_attribute('href'))
            print('********')
            page_config = conf.copy()            
            page_config['department'] = department_name
            page_config['category'] = category_name
            page_config['page'] = category_name
            page_config['page_url'] = cat.find_element(By.CLASS_NAME, 'categoryLinkItem').get_attribute('href')
            page_config['file_name'] = getFilename(page_config)
            page_configs.append(page_config)
#             print(page_config)

In [ ]:
with open("bootscom_urls.json", "w") as outfile:
        json.dump(page_configs, outfile)